In [66]:
from mainnet_launch.data_fetching.alchemy.get_events import fetch_events
from mainnet_launch.constants import ETH_CHAIN


import json

abi = json.loads(
    """[
  {
    "type": "event",
    "name": "TokeLockedEvent",
    "anonymous": false,
    "inputs": [
      { "indexed": true,  "name": "tokeSource",   "type": "address" },
      { "indexed": true,  "name": "account",      "type": "address" },
      { "indexed": false, "name": "numCycles",    "type": "uint256" },
      { "indexed": true,  "name": "currentCycle", "type": "uint256" },
      { "indexed": false, "name": "amount",       "type": "uint256" }
    ]
  },
  {
    "type": "function",
    "name": "getCurrentCycleID",
    "stateMutability": "view",
    "inputs": [],
    "outputs": [
      { "name": "", "type": "uint256" }
    ]
  },
  {
    "type": "function",
    "name": "getDepositInfo",
    "stateMutability": "view",
    "inputs": [
      { "name": "account", "type": "address" }
    ],
    "outputs": [
      { "name": "lockCycle", "type": "uint256" },
      { "name": "lockDuration", "type": "uint256" },
      { "name": "amount", "type": "uint256" }
    ]
  }
]"""
)

staking_contract = ETH_CHAIN.client.eth.contract("0xA374A62DdBd21e3d5716cB04821CB710897c0972", abi=abi)
df = fetch_events(staking_contract.events.TokeLockedEvent, ETH_CHAIN)
df

,event,block,transaction_index,log_index,hash,tokeSource,account,currentCycle,numCycles,amount
0,TokeLockedEvent,20735074,110,426,0xf5bcf35346288c3dac519dca690240e7fbab35088c77...,0xbb42A0Bb44EDe9c170efb8E7EbCA01e21847b700,0xbb42A0Bb44EDe9c170efb8E7EbCA01e21847b700,334,4,10000000000000000
1,TokeLockedEvent,20735086,70,465,0xcd213e188a818a36fcf7dce39f2f7b91ccb6f66a2535...,0x66b416d7DEE321914BA9491eE5E18d8A0aC1Df41,0x66b416d7DEE321914BA9491eE5E18d8A0aC1Df41,334,4,112764691494930538359
2,TokeLockedEvent,20735172,72,237,0x7fe7b8fb7ce0dc69aa75e10928a768154d9c9cdfd2ec...,0xbb42A0Bb44EDe9c170efb8E7EbCA01e21847b700,0xbb42A0Bb44EDe9c170efb8E7EbCA01e21847b700,334,4,10000000000000000
3,TokeLockedEvent,20736112,165,465,0xb8b81f73122c6824e9949bc4c92694916bcdc8646b2e...,0x75E70dB620d5491f69526E22355236f65B46834E,0x75E70dB620d5491f69526E22355236f65B46834E,334,4,2411755216860000000000
4,TokeLockedEvent,20737065,25,178,0xbbf69debb7a874ebb21cd341edd76f173ea93e568ac7...,0x571ba7cEA99697128f3D1d41a44c5e527226E1A3,0x571ba7cEA99697128f3D1d41a44c5e527226E1A3,334,4,75922490980624575471
...,...,...,...,...,...,...,...,...,...,...
2211,TokeLockedEvent,24234345,356,443,0xd3c37dfce4aeb6441250418d121e45f4c3e5889274de...,0xe177E070759EC3493253bD9B3027459BBDABD9FF,0xe177E070759EC3493253bD9B3027459BBDABD9FF,404,16,125482415511259217919
2212,TokeLockedEvent,24238775,50,162,0x88a25185d367a174356041c5a25f7c5957ef023e70f7...,0xb0E1BD3a262DFb2dd87eAb7cc86Ec65c56376861,0xb0E1BD3a262DFb2dd87eAb7cc86Ec65c56376861,404,16,298736107765777987630
2213,TokeLockedEvent,24239106,66,117,0xd88466a8403b65e8d5bcc3fc5d0fb4e5504630214269...,0xb0E1BD3a262DFb2dd87eAb7cc86Ec65c56376861,0xb0E1BD3a262DFb2dd87eAb7cc86Ec65c56376861,404,16,393037678892159600534
2214,TokeLockedEvent,24240977,156,333,0xfd5eefaf546791990204c21c4f84b143e6824e7fa987...,0x06232028c253dA3404cce43A4789dc802a62C846,0x06232028c253dA3404cce43A4789dc802a62C846,404,16,2100122244882823498094


In [67]:
wallets = df['account'].unique().tolist()

from mainnet_launch.data_fetching.get_state_by_block import get_state_by_one_block, ETH_CHAIN, identity_with_bool_success
from multicall import Call

deposit_info_calls = [Call(
    target=staking_contract.address,  # <-- set this to the contract that implements getDepositInfo
    function=["getDepositInfo(address)((uint256,uint256,uint256))", a],
    returns=[(a, identity_with_bool_success)],
) for a in wallets]

state = get_state_by_one_block(deposit_info_calls, ETH_CHAIN.get_block_near_top(), ETH_CHAIN)
state

{'0xbb42A0Bb44EDe9c170efb8E7EbCA01e21847b700': (334, 4, 20000000000000000),
 '0x66b416d7DEE321914BA9491eE5E18d8A0aC1Df41': (399,
  16,
  712615309779107239481),
 '0x75E70dB620d5491f69526E22355236f65B46834E': (0, 0, 0),
 '0x571ba7cEA99697128f3D1d41a44c5e527226E1A3': (334, 4, 784803357228813193825),
 '0x52131AeAf3C6f0D59d81cc00ED5436C1c686A174': (334, 4, 41603037678553516081),
 '0xfd85D71cd9522BFD9B14B5B637b011E2BE2e8c3d': (386,
  16,
  881790025723938398625),
 '0x6F6ed604bc1A64a385978c99310D2fc0758AF29e': (349,
  8,
  2186195280778782703781),
 '0xEe6066b7655657e34f6190629a64F0a8014d3a96': (334, 4, 87968611970782068899),
 '0xf2EfC7b78168abC6A5be99e5624c64f2499d91fC': (379, 4, 291284635338746085456),
 '0x4470e767CA1fDD1469A8E1e5d7592C4097065402': (334, 4, 324240255889274575835),
 '0x9Bc513230b112490527EB6d07594aE9473C4c1EE': (335, 4, 700816579371258829),
 '0x7331059fE92FD0833c7a51C102EA7234F589c5a2': (0, 0, 0),
 '0xD18aB6A2aADA43486b9DFF4f89292553F27B2192': (0, 0, 0),
 '0xB6647caE065c321e

In [80]:
import pandas as pd

current_cycle = staking_contract.functions.getCurrentCycleID().call()
# `state` is typically a dict keyed by whatever you used in `returns` (here: each address),
# with values like (lockCycle, lockDuration, amount) (often as a tuple/list).
rows = []
for account, val in state.items():
    # handle common shapes:
    # - val = (lockCycle, lockDuration, amount)
    # - val = ((lockCycle, lockDuration, amount),)  (extra nesting)
    if val is None:
        lockCycle = lockDuration = amount = None
    else:
        v = val[0] if (isinstance(val, (list, tuple)) and len(val) == 1 and isinstance(val[0], (list, tuple))) else val
        lockCycle, lockDuration, amount = v

    rows.append(
        {"account": account, "lockCycle": lockCycle, "lockDuration": lockDuration, "amount": amount}
    )

state_df = pd.DataFrame(rows).set_index("account")

state_df['total_staked'] = state_df['amount'].apply(lambda x: round (int(x) / 1e18 )if x is not None else 0)
state_df['cycle_unlock'] = state_df['lockCycle'] + state_df['lockDuration']
state_df['days_left'] = (state_df['cycle_unlock'] - current_cycle) * 7

state_df[['total_staked']].sum().apply(lambda x: f"{x:,}")

total_staked    20,609,878
dtype: object

In [81]:
20.6 + 2.81


23.41

In [84]:
state_df[state_df['days_left'] < 0][['total_staked']].sum().apply(lambda x: f"{x:,}")

total_staked    13,350,711
dtype: object

In [85]:
state_df[state_df['days_left'] > 0][['total_staked']].sum().apply(lambda x: f"{x:,}")

total_staked    7,019,747
dtype: object

In [ ]:
state_df['total_staked']

np.int64(20609878)

In [72]:
20,609,878

(20, 609, 878)

In [61]:
active_users

,lockCycle,lockDuration,amount,total_staked,cycle_unlock,days_left,days_left_bin,days_left_label
account,,,,,,,,
0x66b416d7DEE321914BA9491eE5E18d8A0aC1Df41,399,16,712615309779107239481,713,415,77,56,56-83 days
0x123cC4AFA59160C6328C0152cf333343F510e5A3,396,16,76394740031096858770,76,412,56,56,56-83 days
0xf5042b588f98Fd57f0D370Fce7BF6c869597e995,394,16,19414383249190046994281,19414,410,42,28,28-55 days
0xBbc3981Cb8116477B7Ba26f347Cf6B07e09EDF6C,398,16,13159560318781236509845,13160,414,70,56,56-83 days
0x316B98C6df382BD218186D0766966459A199Dd04,393,16,10013008568190497941238,10013,409,35,28,28-55 days
...,...,...,...,...,...,...,...,...
0x333499Df348BDF759b6f9F86d7485418b7b58233,401,16,13739790197310314881,14,417,91,84,84-111 days
0xe625759D199F3f1B7aEC295cc69796762d3F21C0,401,4,3770339349830247628178,3770,405,7,0,0-27 days
0xCE777492fFb5183A890A97bb43f29f8A21B685DF,402,16,528551309089694236327,529,418,98,84,84-111 days


In [62]:
active_users.to_csv('stoke_active_users.csv')

In [64]:
active_users['total_staked'].sum()

np.int64(7019747)

In [47]:
# Create bins for days_left in 28-day chunks
active_users['days_left_bin'] = (active_users['days_left'] // 28) * 28
active_users['days_left_label'] = active_users['days_left_bin'].astype(str) + '-' + (active_users['days_left_bin'] + 27).astype(str) + ' days'

# Display the distribution
active_users.groupby('days_left_label')['total_staked'].sum().apply(lambda x: f"{x:,}")

days_left_label
0-27 days         611,199
112-139 days      509,368
28-55 days      1,903,261
56-83 days        596,105
84-111 days     3,399,814
Name: total_staked, dtype: object

In [49]:
active_users

,lockCycle,lockDuration,amount,total_staked,cycle_unlock,days_left,days_left_bin,days_left_label
account,,,,,,,,
0x66b416d7DEE321914BA9491eE5E18d8A0aC1Df41,399,16,712615309779107239481,713,415,77,56,56-83 days
0x123cC4AFA59160C6328C0152cf333343F510e5A3,396,16,76394740031096858770,76,412,56,56,56-83 days
0xf5042b588f98Fd57f0D370Fce7BF6c869597e995,394,16,19414383249190046994281,19414,410,42,28,28-55 days
0xBbc3981Cb8116477B7Ba26f347Cf6B07e09EDF6C,398,16,13159560318781236509845,13160,414,70,56,56-83 days
0x316B98C6df382BD218186D0766966459A199Dd04,393,16,10013008568190497941238,10013,409,35,28,28-55 days
...,...,...,...,...,...,...,...,...
0x333499Df348BDF759b6f9F86d7485418b7b58233,401,16,13739790197310314881,14,417,91,84,84-111 days
0xe625759D199F3f1B7aEC295cc69796762d3F21C0,401,4,3770339349830247628178,3770,405,7,0,0-27 days
0xCE777492fFb5183A890A97bb43f29f8A21B685DF,402,16,528551309089694236327,529,418,98,84,84-111 days


In [50]:
active_users.groupby('lockDuration')['total_staked'].sum().apply(lambda x: f"{x:,}")

lockDuration
4         4,069
12      249,524
16    6,766,154
Name: total_staked, dtype: object

In [36]:
df["block"].quantile([0.6]).values[0].astype(int)  # 246 days

np.int64(22415101)

In [37]:
df["toke"] = df["amount"].apply(lambda x: int(x) / 1e18)
df["months"] = df["numCycles"].apply(lambda x: x / 4)
print("All time percent of toke staked events by how many months")
df["months"].value_counts(normalize=True).apply(lambda x: round((x * 100), 2))

All time percent of toke staked events by how many months


months
4.0    65.22
1.0    27.44
2.0     3.24
3.0     2.55
1.5     1.37
0.5     0.18
Name: proportion, dtype: float64

In [38]:
print("All time percent of toke staked by total toke staked")
(df.groupby("months")["toke"].sum() / df["toke"].sum()).apply(lambda x: round((x * 100), 2))

All time percent of toke staked by total toke staked


months
0.5     0.06
1.0    28.16
1.5     0.16
2.0    15.96
3.0     1.45
4.0    54.21
Name: toke, dtype: float64

In [42]:
import plotly.express as px
import plotly.io as pio

pio.templates.default = None

px.scatter(df, x="block", y="months", size="toke", title="lock duration over time")

In [45]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.templates.default = None

bin_size = 500_000

dff = df.copy()
dff["block_bin_start"] = (dff["block"] // bin_size) * bin_size
dff["block_bin_label"] = dff["block_bin_start"].astype(str) + "–" + (dff["block_bin_start"] + bin_size - 1).astype(str)

# aggregate TOKE per (block bin, months) so we can color by months
agg = (
    dff.groupby(["block_bin_start", "block_bin_label", "months"], as_index=False)["toke"]
    .sum()
    .sort_values(["block_bin_start", "months"])
)

fig = px.bar(
    agg,
    x="block_bin_label",
    y="toke",
    color="months",
    title="TOKE locked per 500k-block window (colored by lock duration)",
    labels={"block_bin_label": "Block range (500k)", "toke": "TOKE locked", "months": "Lock duration (months)"},
)

fig.update_xaxes(type="category", tickangle=-45)
fig.show()

In [47]:
df["account"].value_counts()

account
0xB6647caE065c321e75775B50a25fd54e9ac5C436    56
0xECfECF375160D2E3d1aB27A31a161D955C7650Cb    40
0xE2Aee7E1251D4690297F654822eB6698638cBC59    39
0x133D93566f9699B3Af46fE150daA8a67a9563ED6    29
0xC0810E5f17915dFe97b57947A3d84094572689ac    28
                                              ..
0x178689A02a03F6fdDBc20E40Fe90487906AEe39e     1
0xA410BBE4765b8db0cb8dC352278619442161083D     1
0x193991827E291599A262e7fa7D212ff1Ae31D110     1
0x387b5574bd861587fcf2bE1951c857897461240F     1
0x01485ef10715F5B96BeFfA5552B0642E32C9aef6     1
Name: count, Length: 685, dtype: int64

In [51]:
guy = df[df["account"] == "0xB6647caE065c321e75775B50a25fd54e9ac5C436"]

px.scatter(guy, x="currentCycle", y="numCycles", size="toke")

In [55]:
import json
import pandas as pd

from mainnet_launch.data_fetching.alchemy.get_events import fetch_events
from mainnet_launch.constants import ETH_CHAIN

WALLET = "0xA374A62DdBd21e3d5716cB04821CB710897c0972"
TOKEN  = "0x2e9d63788249371f1DFC918a52f8d799F4a38C94"

erc20_abi = json.loads("""
[
  {
    "anonymous": false,
    "inputs": [
      {"indexed": true, "name": "from", "type": "address"},
      {"indexed": true, "name": "to",   "type": "address"},
      {"indexed": false,"name": "value","type": "uint256"}
    ],
    "name": "Transfer",
    "type": "event"
  }
]
""")

token_contract = ETH_CHAIN.client.eth.contract(
    address=TOKEN,
    abi=erc20_abi,
)

wallet_cs = ETH_CHAIN.client.toChecksumAddress(WALLET)

# Optional: set a start_block to reduce scan cost/time
# start_block = 0
start_block = None
end_block = None

df_from = fetch_events(
    event=token_contract.events.Transfer,
    chain=ETH_CHAIN,
    start_block=start_block,
    end_block=end_block,
    argument_filters={"from": wallet_cs},
)

df_to = fetch_events(
    event=token_contract.events.Transfer,
    chain=ETH_CHAIN,
    start_block=start_block,
    end_block=end_block,
    argument_filters={"to": wallet_cs},
)

df = pd.concat([df_from, df_to], ignore_index=True)

# Deduplicate in case of any overlap or repeated retrieval
# (columns depend on your decode_logs; adjust if needed)
dedupe_cols = [c for c in ["transactionHash", "logIndex"] if c in df.columns]
if dedupe_cols:
    df = df.drop_duplicates(subset=dedupe_cols)

# Normalize / convenience fields if your decoded df uses different casing
# Common patterns I’ve seen: 'args_from'/'args_to' or 'from'/'to'
for a, b in [("from", "from"), ("to", "to"), ("value", "value")]:
    if a not in df.columns and f"args_{b}" in df.columns:
        df[a] = df[f"args_{b}"]

df["direction"] = df.apply(
    lambda r: "out" if str(r.get("from", "")).lower() == WALLET.lower()
    else ("in" if str(r.get("to", "")).lower() == WALLET.lower() else "other"),
    axis=1,
)

# Optional sorting if you have blockNumber / transactionIndex
for sort_col in ["blockNumber", "transactionIndex", "logIndex"]:
    if sort_col not in df.columns:
        break
else:
    df = df.sort_values(["blockNumber", "transactionIndex", "logIndex"])

df


,event,block,transaction_index,log_index,hash,from,to,value,direction
0,Transfer,20734753,73,145,0x261ee58aab5ad8c81a1b6b3cf3e4b48457b2f1c49c90...,0xA374A62DdBd21e3d5716cB04821CB710897c0972,0xaAD4A4a165Ac449b59Ea68d81B97f1Eaf314D501,237688136463737661624,out
1,Transfer,20741836,43,60,0x86f106a351a95833d05f8b921981f91e4168e54707de...,0xA374A62DdBd21e3d5716cB04821CB710897c0972,0x3c8e741c0a2Cb4b8d5cBB1ead482CFDF87FDd66F,37968611970782068858,out
2,Transfer,20781258,45,243,0xbbc68986fa367b2c5f924e2772ac332723c2e46eec4b...,0xA374A62DdBd21e3d5716cB04821CB710897c0972,0xD99323742240C92175B31013C4687c549e1b7971,95385510001676158810,out
3,Transfer,20785194,114,191,0x836ddff0c886d192c17044e8c430b42a0c41ba6ded0c...,0xA374A62DdBd21e3d5716cB04821CB710897c0972,0x22F2ea96BC91F1Eb9d232B60b3DA9c2F04776905,69792030268712424600696,out
4,Transfer,20789206,74,459,0x56cb4547a78d6a1566337eab00743cf2577d5b14b655...,0xA374A62DdBd21e3d5716cB04821CB710897c0972,0x1Da376e38447066d42bcc01552DbD19e48544AE6,1586269502035984183093,out
...,...,...,...,...,...,...,...,...,...
2494,Transfer,24234345,356,441,0xd3c37dfce4aeb6441250418d121e45f4c3e5889274de...,0xe177E070759EC3493253bD9B3027459BBDABD9FF,0xA374A62DdBd21e3d5716cB04821CB710897c0972,125482415511259217919,in
2495,Transfer,24238775,50,160,0x88a25185d367a174356041c5a25f7c5957ef023e70f7...,0xb0E1BD3a262DFb2dd87eAb7cc86Ec65c56376861,0xA374A62DdBd21e3d5716cB04821CB710897c0972,298736107765777987630,in
2496,Transfer,24239106,66,115,0xd88466a8403b65e8d5bcc3fc5d0fb4e5504630214269...,0xb0E1BD3a262DFb2dd87eAb7cc86Ec65c56376861,0xA374A62DdBd21e3d5716cB04821CB710897c0972,393037678892159600534,in
2497,Transfer,24240977,156,331,0xfd5eefaf546791990204c21c4f84b143e6824e7fa987...,0x06232028c253dA3404cce43A4789dc802a62C846,0xA374A62DdBd21e3d5716cB04821CB710897c0972,2100122244882823498094,in


In [58]:
df['size'] = df['value'].apply(lambda x: round (int(x) / 1e18, 2 )if x is not None else 0)
df

,event,block,transaction_index,log_index,hash,from,to,value,direction,size
0,Transfer,20734753,73,145,0x261ee58aab5ad8c81a1b6b3cf3e4b48457b2f1c49c90...,0xA374A62DdBd21e3d5716cB04821CB710897c0972,0xaAD4A4a165Ac449b59Ea68d81B97f1Eaf314D501,237688136463737661624,out,237.69
1,Transfer,20741836,43,60,0x86f106a351a95833d05f8b921981f91e4168e54707de...,0xA374A62DdBd21e3d5716cB04821CB710897c0972,0x3c8e741c0a2Cb4b8d5cBB1ead482CFDF87FDd66F,37968611970782068858,out,37.97
2,Transfer,20781258,45,243,0xbbc68986fa367b2c5f924e2772ac332723c2e46eec4b...,0xA374A62DdBd21e3d5716cB04821CB710897c0972,0xD99323742240C92175B31013C4687c549e1b7971,95385510001676158810,out,95.39
3,Transfer,20785194,114,191,0x836ddff0c886d192c17044e8c430b42a0c41ba6ded0c...,0xA374A62DdBd21e3d5716cB04821CB710897c0972,0x22F2ea96BC91F1Eb9d232B60b3DA9c2F04776905,69792030268712424600696,out,69792.03
4,Transfer,20789206,74,459,0x56cb4547a78d6a1566337eab00743cf2577d5b14b655...,0xA374A62DdBd21e3d5716cB04821CB710897c0972,0x1Da376e38447066d42bcc01552DbD19e48544AE6,1586269502035984183093,out,1586.27
...,...,...,...,...,...,...,...,...,...,...
2494,Transfer,24234345,356,441,0xd3c37dfce4aeb6441250418d121e45f4c3e5889274de...,0xe177E070759EC3493253bD9B3027459BBDABD9FF,0xA374A62DdBd21e3d5716cB04821CB710897c0972,125482415511259217919,in,125.48
2495,Transfer,24238775,50,160,0x88a25185d367a174356041c5a25f7c5957ef023e70f7...,0xb0E1BD3a262DFb2dd87eAb7cc86Ec65c56376861,0xA374A62DdBd21e3d5716cB04821CB710897c0972,298736107765777987630,in,298.74
2496,Transfer,24239106,66,115,0xd88466a8403b65e8d5bcc3fc5d0fb4e5504630214269...,0xb0E1BD3a262DFb2dd87eAb7cc86Ec65c56376861,0xA374A62DdBd21e3d5716cB04821CB710897c0972,393037678892159600534,in,393.04
2497,Transfer,24240977,156,331,0xfd5eefaf546791990204c21c4f84b143e6824e7fa987...,0x06232028c253dA3404cce43A4789dc802a62C846,0xA374A62DdBd21e3d5716cB04821CB710897c0972,2100122244882823498094,in,2100.12


In [ ]:
active_users['']